# Interacting with Large Language Models using Semantic Kernel

This notebook demonstrates how to interact with large language models (LLMs) using the Microsoft Semantic Kernel in .NET.

**Objectives:**
- Understand how to set up Semantic Kernel for LLM interaction in .NET.
- Learn to configure and connect to different model providers (OpenAI, Azure OpenAI, GitHub models).
- Use model parameters to customize model behavior.
- Send prompts to LLMs and receive responses.
- Use prompt templates and kernel arguments for dynamic, reusable prompts.

## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.55.0

**Step 2**: Read environment variables

  In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.


In [21]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3**: Instantiate the Kernel

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models.

In [22]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client);

// Get the chat completion service
Kernel kernel = builder.Build();

## Calling LLMs

This section demonstrates how to call different LLMs using the Semantic Kernel.

**Step 1**: Call the Kernel

In this step, we send a prompt to the Semantic Kernel and receive a response from the AI model. 

The model uses the training data to generate a response based on the input prompt. In the next example, you will see how to use your own data as the context for the model.

In [24]:
using Microsoft.SemanticKernel.Connectors.OpenAI;

//call the kernel to get a response
var prompt = @"I want to buy a new bike for daily commuting to work and weekend rides. 
My preferences are a lightweight frame, quality suspension, and reliable disc brakes. 
Could you recommend a few suitable options and explain the advantages and disadvantages of each for my needs?";
var response = await kernel.InvokePromptAsync(prompt);
Console.WriteLine($"Response: {response}");

Response: Of course! Based on your preferences for lightweight construction, quality suspension, and reliable disc brakes, here are a few bike types and specific models to consider for both daily commuting and weekend rides. I'll outline advantages and disadvantages for each to help you decide:

---

### 1. **Hybrid Bike**
**Example Models**: Trek FX Sport 4, Cannondale Quick Disc 3, Giant Escape Disc Series

#### Advantages:
- **Lightweight Frame**: Most hybrid bikes are made from aluminum or carbon, ensuring a lightweight build for quick acceleration and easy handling.
- **Versatility**: Designed for smooth tarmac and light off-roading, hybrids are ideal for commuting and casual weekend rides.
- **Flat Handlebars**: Offer comfortable, upright positioning for city riding.
- **Reliable Disc Brakes**: Equipped with hydraulic or mechanical disc brakes for consistent stopping power in all weather.
- **Low Maintenance**: Simpler drivetrains compared to mountain bikes, making them easy to m

**Step 2**: Use Prompt Templates and Kernel Arguments

Prompt templates allow you to create reusable prompts with placeholders for dynamic values. Kernel arguments let you pass values to these placeholders at runtime, making your prompts flexible and powerful.

In [27]:
// Define a prompt template with a placeholder
string template = "Can you tell me about some options for {{$bike_type}} bikes?";
// Create a function from the prompt template
var templateFunction = kernel.CreateFunctionFromPrompt(template);

// Prepare kernel arguments
var arguments = new KernelArguments
{
    ["bike_type"] = "Hybrid"
};

// Call the kernel with the function and arguments
var response = await kernel.InvokeAsync(templateFunction, arguments);
Console.WriteLine($"Response: {response}");

Response: Certainly! Hybrid bikes are a fantastic choice for riders who want versatility, comfort, and performance for both casual rides and commuting. They combine features of road bikes and mountain bikes, making them ideal for a variety of terrains, from city streets to light trails. Below, you'll find a list of hybrid bikes across different price ranges to suit your needs and preferences:

---

### **1. TREK Verve Series**
- **Key Features:**
  - Lightweight aluminum frame.
  - Upright geometry for added comfort.
  - Ergonomic grips and a plush saddle.
  - Equipped with disc brakes for all-weather stopping power.
  - Ideal for fitness rides and city commuting.
- **Why Choose It?** Perfect for casual riders who prioritize comfort and reliability.

---

### **2. Specialized Sirrus**
- **Key Features:**
  - Sleek and lightweight aluminum frame.
  - Internal cable routing for cleaner aesthetics.
  - Suspension seat post for added comfort (on select models).
  - Some versions include ca

**Step 3**: Try with another bike

You can reuse the same prompt template and function with different arguments to get answers for other countries.

In [28]:
arguments["bike_type"] = "Road";
response = await kernel.InvokeAsync(templateFunction, arguments);
Console.WriteLine($"Response: {response}");

Response: Sure! Road bikes are designed for speed and efficiency on paved surfaces, making them an excellent choice for recreational riders, fitness enthusiasts, and competitive cyclists. There are many great options available across various budgets and styles. Here’s a general breakdown of some popular road bike categories and models that might suit your needs:

---

### **1. Entry-Level Road Bikes**
These are ideal for beginners or those looking for an affordable way to get into road biking.

- **Specialized Allez**
  - Lightweight aluminum frame with smooth handling and an accessible price point.
  - Offers a confident ride and quality Shimano components.

- **Trek Domane AL 2**
  - Endurance-focused geometry for comfort on longer rides.
  - Made with a lightweight aluminum frame and features Shimano Claris drivetrain.

- **Giant Contend AR**
  - A versatile bike that’s comfortable for long-distance rides and compatible with wider tires.
  - Aluminum frame with carbon fork for a smo

## Using System Prompt and User Prompt

Semantic Kernel allows you to provide both a system prompt (to guide the model's behavior) and a user prompt (the actual user input). This is useful for customizing the assistant's persona or instructions for a conversation.

In [31]:
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Enhanced Example: Use a richer system prompt and inject dynamic bike data

// Example bike data (in a real scenario, this could come from a database or API)
var bikeData = @"
| Model           | Store             | Description                                             | Tags                        |
|-----------------|-------------------|---------------------------------------------------------|-----------------------------|
| Mountain Bikes  | Contoso Bike Store| Durable, great for off-road and trails.                 | mountain, off-road, trails  |
| Road Bikes      | Contoso Bike Store| Lightweight, fast, ideal for pavement.                  | road, lightweight, pavement |
| Hybrid Bikes    | Contoso Bike Store| Versatile, comfortable, suitable for city and light trails.| hybrid, city, comfort    |
| Electric Bikes  | Contoso Bike Store| Assisted pedaling, perfect for longer commutes or hilly areas.| electric, commute, hills |
";

// Compose the system prompt with bike data injected
var systemPrompt = $@"You are a friendly, knowledgeable assistant for Contoso Bike Store.
Answer questions about bikes with enthusiasm, clarity, and practical advice. If you are unsure, say 'I don't know.'
Always provide helpful tips, context, and encourage follow-up questions.
Maintain an approachable, supportive tone, and focus on making the customer feel welcome and informed.

Your response will be based on the following bike data:

{{$bikeData}}
";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddAssistantMessage("Hi there! Welcome to Contoso Bike Store. How can I assist you today?");
chatHistory.AddUserMessage("What is the best bike for city commuting?");

var arguments = new KernelArguments();
arguments["bikeData"] = bikeData;

var chat = kernel.GetRequiredService<IChatCompletionService>(); 
var chatResponse = await chat.GetChatMessageContentAsync(chatHistory);
Console.WriteLine($"Response: {chatResponse}");

Response: Great question! For city commuting, you’ll want a bike that’s comfortable, efficient, and well-suited for urban terrain. Here are some factors to consider and suggestions:

### **Best Options for City Commuting:**
1. **Hybrid Bikes**  
   - These are super versatile and combine the speed of a road bike with the comfort of a mountain bike. They’re perfect for commuting on paved roads and navigating through traffic. Look for options with upright handlebars for better visibility and comfort.
   - Key benefits: Lightweight, easy to ride, and great for short-to-medium distances.

2. **Electric Bikes (E-Bikes)**  
   - If you want to commute without breaking a sweat or need an extra boost for longer rides, e-bikes are an awesome choice. They come with pedal-assist features that make hills and long stretches a breeze.
   - Key benefits: Time-saving, energy-efficient, and an eco-friendly alternative to cars.

3. **Commuter or Urban Bikes**  
   - These bikes are specifically designed

## Model Parameters

Large language models expose several parameters that control the behavior and quality of their responses. 

**Common Model Parameters:**
- **Temperature:** Controls the randomness of the output. Lower values (e.g., 0.2) make the output more focused and deterministic, while higher values (e.g., 0.8) make it more creative and random.
- **MaxTokens:** Limits the maximum number of tokens (words or word pieces) in the response. Useful for controlling the length of the output.
- **TopP:** Makes the model pick from the most likely words until their total chance adds up to TopP (like 0.5). Lower values make the answer more focused.

> **Note:** The exact set of parameters and their effects may vary depending on the model provider (OpenAI, Azure OpenAI, etc.).

In [32]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
 
#pragma warning disable SKEXP0001
var openAiPromptSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 100,
    Temperature = 0.5,
    TopP = 0.8
};
#pragma warning restore SKEXP0001

var chat = kernel.GetRequiredService<IChatCompletionService>(); 
// Create a prompt that will trigger the function call
string prompt = "Write a short poem about the ocean.";
var response = await chat.GetChatMessageContentAsync("Write a short poem about the ocean.", openAiPromptSettings, kernel);
Console.WriteLine($"Response: {response}");

Response: Beneath the sky so vast and blue,  
The ocean whispers, deep and true.  
Its waves compose a timeless song,  
A rhythm ancient, fierce, and strong.  

The sun ignites its sparkling crest,  
While moonlight soothes its restless breast.  
It holds the secrets of the deep,  
Where dreams and shadows softly sleep.  

A boundless heart, both wild and free,  
The ocean calls eternally.  
Its voice, a hymn of endless grace


You can adjust these parameters to experiment with different response styles and lengths. Try changing the temperature or max tokens and observe how the model's output changes.